# 🖥️ CLI Examples - PTOF Analysis System

Questo notebook contiene tutti gli snippet di codice per utilizzare gli script CLI del sistema di analisi PTOF.

**⚙️ Configurazione**: Tutti i comandi utilizzano il virtual environment `.venv` del progetto.

**📂 Struttura Directory**:
- `ptof_inbox/` → PDF da analizzare (INSERISCI QUI)
- `ptof_processed/` → PDF archiviati automaticamente

## 📑 Indice
1. [🚀 **Workflow Automatico**](#workflow-automatico) ⭐ PRIORITÀ
2. [🔗 Analisi Multi-Agente Batch](#multi-agent-batch)
3. [☁️ Cloud Agent](#cloud-agent)
4. [🔍 Analisi e Revisione](#analysis-review)
5. [🛠️ Utility](#utilities)
6. [🤖 Automazione Background](#background-automation)
7. [📊 Diagnostica](#diagnostics)

---
## 🚀 Workflow Automatico {#workflow-automatico}

### ⭐ Pipeline Completa: Inbox → Processed

**Cosa fa**:
1. 📥 Legge PDF da `ptof_inbox/`
2. 📝 Converte PDF → Markdown
3. 🤖 Analizza con pipeline multi-agente
4. 📦 Sposta PDF in `ptof_processed/batch_TIMESTAMP/`
5. 📊 Aggiorna CSV dashboard

In [4]:
%%bash
# WORKFLOW COMPLETO - Inbox → Processed
cd /Users/danieledragoni/git/LIste
source .venv/bin/activate

echo "🚀 WORKFLOW AUTOMATICO PTOF"
echo "📥 Input: ptof_inbox/"
echo "✅ Output: ptof_processed/batch_TIMESTAMP/"
echo "========================================"

python workflow_ptof.py 2>&1 | tee logs/workflow_ptof.log

echo "========================================"
echo "✅ Workflow completato!"
echo "📋 Log: logs/workflow_ptof.log"

🚀 WORKFLOW AUTOMATICO PTOF
📥 Input: ptof_inbox/
✅ Output: ptof_processed/batch_TIMESTAMP/
2025-12-21 21:54:28,853 - INFO - 📂 Directory verificata: ptof_inbox/
2025-12-21 21:54:28,854 - INFO - 📂 Directory verificata: ptof_processed/
2025-12-21 21:54:28,854 - INFO - 📂 Directory verificata: ptof_md/
2025-12-21 21:54:28,854 - INFO - 📂 Directory verificata: analysis_results/
2025-12-21 21:54:28,854 - INFO - 📂 Directory verificata: logs/
2025-12-21 21:54:28,854 - INFO - ================================================================================
2025-12-21 21:54:28,854 - INFO - 🚀 AVVIO WORKFLOW PTOF COMPLETO
2025-12-21 21:54:28,854 - INFO - 🕐 Timestamp: 2025-12-21 21:54:28
2025-12-21 21:54:28,854 - INFO - ================================================================================
2025-12-21 21:54:28,855 - INFO - 
📊 STATO INIZIALE:
2025-12-21 21:54:28,855 - INFO -   PDF in inbox: 2
2025-12-21 21:54:28,855 - INFO -   PDF processati: 0
2025-12-21 21:54:28,855 - INFO -   File Markdown: 

### Verifica Stato Directory

In [ ]:
%%bash
cd /Users/danieledragoni/git/LIste

echo "📊 STATO DIRECTORY"
echo "========================================"
echo "📥 Inbox PDF: $(find ptof_inbox -name '*.pdf' 2>/dev/null | wc -l)"
echo "✅ Processed PDF: $(find ptof_processed -name '*.pdf' 2>/dev/null | wc -l)"
echo "📝 Markdown: $(find ptof_md -name '*.md' 2>/dev/null | wc -l)"
echo "📊 Analisi JSON: $(find analysis_results -name '*.json' 2>/dev/null | wc -l)"
echo "========================================"

---
## 🔗 Analisi Multi-Agente Batch {#multi-agent-batch}

### Pipeline Multi-Agente per Cartella Completa
Processa tutti i file PTOF in una cartella usando l'architettura multi-agente (Analyst → Reviewer → Refiner → Synthesizer).

In [ ]:
%%bash
# Pipeline Multi-Agente - Batch completo su cartella ptof_md/
cd /Users/danieledragoni/git/LIste
source .venv/bin/activate

echo "🚀 Avvio Pipeline Multi-Agente Batch"
echo "📂 Directory: ptof_md/"
echo "🤖 Agenti: Analyst → Reviewer → Refiner"
echo "========================================"

python app/agentic_pipeline.py 2>&1 | tee logs/agentic_pipeline.log

echo "========================================"
echo "✅ Pipeline completata!"
echo "📋 Log: logs/agentic_pipeline.log"

### Pipeline Multi-Agente - Singolo File con Logging

In [ ]:
# Pipeline Multi-Agente - Modalità singolo file con logging dettagliato
import sys
import os
import logging
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/single_analysis.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Aggiungi path progetto
sys.path.insert(0, '/Users/danieledragoni/git/LIste')

from app.agentic_pipeline import (
    process_single_ptof,
    AnalystAgent,
    ReviewerAgent,
    RefinerAgent,
    SynthesizerAgent
)

logger.info("🚀 Inizializzazione agenti multi-agente")

# Inizializza agenti
analyst = AnalystAgent()
reviewer = ReviewerAgent()
refiner = RefinerAgent()
synthesizer = SynthesizerAgent()

logger.info("✅ Agenti inizializzati con successo")

# Status callback con logging
def status_callback(msg):
    logger.info(f"[PIPELINE] {msg}")

# Processa un singolo file PTOF
md_file = "ptof_md/MIIS08900V.md"  # Modifica con il tuo file
logger.info(f"📄 Processando file: {md_file}")

start_time = datetime.now()

result = process_single_ptof(
    md_file=md_file,
    analyst=analyst,
    reviewer=reviewer,
    refiner=refiner,
    synthesizer=synthesizer,
    status_callback=status_callback
)

elapsed = (datetime.now() - start_time).total_seconds()

logger.info("="*80)
logger.info(f"✅ Analisi completata in {elapsed:.1f} secondi!")
logger.info(f"📊 Risultato keys: {list(result.keys()) if result else 'None'}")
logger.info(f"📋 Log salvato in: logs/single_analysis.log")

if result:
    metadata = result.get('metadata', {})
    logger.info(f"🏫 Scuola: {metadata.get('denominazione', 'N/A')}")
    logger.info(f"📍 Comune: {metadata.get('comune', 'N/A')}")

---
## ☁️ Cloud Agent {#cloud-agent}

### Analisi Cloud con Logging

In [ ]:
# Cloud Agent - Analisi completa con logging dettagliato
import sys
import logging
from datetime import datetime

sys.path.insert(0, '/Users/danieledragoni/git/LIste')

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/cloud_analysis.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

from src.processing.cloud_review import review_ptof_with_cloud, load_api_config

logger.info("☁️ Inizializzazione Cloud Agent")

# Carica configurazione API dal file JSON
config = load_api_config()
provider = config.get('default_provider', 'gemini')
api_key_field = f'{provider}_api_key'

logger.info(f"🔐 Provider: {provider}")
logger.info(f"🔑 API Key: {'✅ Configurata' if config.get(api_key_field) else '❌ Mancante'}")

# Carica il contenuto del file MD
md_file = 'ptof_md/MIIS08900V.md'
logger.info(f"📄 Caricando file: {md_file}")

with open(md_file, 'r', encoding='utf-8') as f:
    text = f.read()

logger.info(f"📏 Dimensione documento: {len(text):,} caratteri")

# Esegui analisi completa
start_time = datetime.now()
logger.info("🚀 Avvio analisi cloud...")

result = review_ptof_with_cloud(
    md_content=text,
    provider=provider,
    api_key=config.get(api_key_field),
    model="gemini-2.0-flash-exp" if provider == 'gemini' else "google/gemini-2.0-flash-exp:free",
    school_id="MIIS08900V"
)

elapsed = (datetime.now() - start_time).total_seconds()

logger.info("="*80)
logger.info(f"✅ Analisi completata in {elapsed:.1f} secondi!")

if result:
    metadata = result.get('metadata', {})
    logger.info(f"🏫 School ID: {metadata.get('school_id', 'N/A')}")
    logger.info(f"📛 Denominazione: {metadata.get('denominazione', 'N/A')}")
    logger.info(f"📍 Comune: {metadata.get('comune', 'N/A')}")
    logger.info(f"🗺️ Area: {metadata.get('area_geografica', 'N/A')}")
    
logger.info(f"📋 Log salvato in: logs/cloud_analysis.log")

---
## 🔍 Analisi e Revisione {#analysis-review}

### Rebuild CSV con Logging

In [ ]:
%%bash
# Rebuild CSV - Ricostruzione indice da file JSON
cd /Users/danieledragoni/git/LIste
source .venv/bin/activate

echo "📊 Ricostruzione CSV da file JSON"
echo "📂 Input: analysis_results/*.json"
echo "📄 Output: data/analysis_summary.csv"
echo "========================================"

python src/processing/rebuild_csv.py 2>&1 | tee logs/rebuild_csv.log

echo "========================================"
echo "✅ CSV ricostruito!"
echo "📋 Log: logs/rebuild_csv.log"

---
## 🛠️ Utility {#utilities}

### Conversione PDF → Markdown con Logging

In [ ]:
%%bash
# Conversione PDF → Markdown
cd /Users/danieledragoni/git/LIste
source .venv/bin/activate

echo "🔄 Conversione PDF → Markdown"
echo "📂 Input: ptof/*.pdf"
echo "📂 Output: ptof_md/*.md"
echo "========================================"

python src/processing/convert_pdfs_to_md.py 2>&1 | tee logs/pdf_conversion.log

echo "========================================"
echo "✅ Conversione completata!"
echo "📋 Log: logs/pdf_conversion.log"

---
## 🤖 Automazione Background {#background-automation}

### Background Fixer - CLI con Logging

In [ ]:
%%bash
# Background Fixer - Correzione automatica anomalie
cd /Users/danieledragoni/git/LIste
source .venv/bin/activate

echo "🛠️ Background Fixer - Avvio"
echo "📂 Input: data/review_flags.json"
echo "📂 Output: analysis_results/"
echo "🤖 Modello: qwen2.5-coder:7b (Ollama)"
echo "========================================"

python run_fixer.py 2>&1 | tee logs/background_fixer.log

echo "========================================"
echo "✅ Correzione completata!"
echo "📋 Log: logs/background_fixer.log"

---
## 📊 Diagnostica {#diagnostics}

### Verifica Stato Sistema Completo

In [ ]:
# System Status Check - Diagnostica completa con logging
import os
import json
import logging
from glob import glob
from datetime import datetime

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/system_diagnostics.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

logger.info("="*80)
logger.info("📊 DIAGNOSTICA SISTEMA PTOF ANALYSIS")
logger.info(f"🕐 Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
logger.info("="*80)

# Check review flags
logger.info("\n🚩 ANOMALIE RILEVATE")
if os.path.exists("data/review_flags.json"):
    with open("data/review_flags.json", 'r') as f:
        flags = json.load(f)
    logger.info(f"  File con anomalie: {len(flags)}")
else:
    logger.info("  Nessun file di anomalie trovato")

# Count files
logger.info("\n📄 FILE SISTEMA")
json_files = glob("analysis_results/*.json")
logger.info(f"  File analisi JSON: {len(json_files)}")

md_files = glob("ptof_md/*.md")
logger.info(f"  File Markdown: {len(md_files)}")

pdf_inbox = glob("ptof_inbox/*.pdf")
logger.info(f"  PDF in inbox: {len(pdf_inbox)}")

# API Configuration
logger.info("\n🔐 CONFIGURAZIONE API")
if os.path.exists("data/api_config.json"):
    with open("data/api_config.json", 'r') as f:
        api_config = json.load(f)
    logger.info(f"  Default Provider: {api_config.get('default_provider', 'N/A')}")
    logger.info(f"  Gemini API: {'✅' if api_config.get('gemini_api_key') else '❌'}")
    logger.info(f"  OpenRouter API: {'✅' if api_config.get('openrouter_api_key') else '❌'}")

logger.info("\n="*80)
logger.info("✅ Diagnostica completata!")
logger.info("📋 Log: logs/system_diagnostics.log")

---
## 📝 Note

### Best Practices

- ✅ **Virtual Environment**: Tutti i comandi usano `.venv`
- ✅ **Logging**: Ogni operazione salva log in `logs/`
- ✅ **Workflow Automatico**: Usa `workflow_ptof.py` per processare nuovi PDF
- ⚠️ **API Keys**: Configurate in `data/api_config.json`
- ⚠️ **Ollama**: Deve essere in esecuzione per pipeline multi-agente

### Directory

- `ptof_inbox/` - PDF da analizzare
- `ptof_processed/` - PDF archiviati
- `ptof_md/` - Markdown generati
- `analysis_results/` - JSON analisi
- `logs/` - File di log